# Baseline Model: Arrhenius Kinetics + MLP with BatchNorm

This notebook implements:
1. Physics-informed features (1/T, ln(t), interaction)
2. MLP with BatchNorm, Dropout, Sigmoid output
3. HuberLoss for robustness
4. Symmetry TTA for mixed solvents
5. Bagging (3 models for speed)

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

# Check GPU
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

torch.set_default_dtype(torch.double)

CUDA available: True
GPU: NVIDIA H100 80GB HBM3
Memory: 85.0 GB


In [2]:
# Data loading utilities - adapted for local paths
DATA_PATH = '/home/data'

INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load spange descriptors
SPANGE_DF = load_features('spange_descriptors')
print(f"Loaded spange descriptors: {SPANGE_DF.shape}")

Loaded spange descriptors: (26, 13)


In [3]:
# Kinetic Featurizer with Arrhenius features
class KineticMixingFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.featurizer = SPANGE_DF
        # Features: 2 numeric + 3 kinetic + 13 spange = 18
        self.feats_dim = self.featurizer.shape[1] + 2 + 3

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        
        # Arrhenius kinetic features
        temp_c = X_vals[:, 1:2]  # Temperature in Celsius
        time_m = X_vals[:, 0:1]  # Residence time in minutes
        
        temp_k = temp_c + 273.15  # Convert to Kelvin
        inv_temp = 1000.0 / temp_k  # Inverse temperature (scaled)
        log_time = np.log(time_m + 1e-6)  # Log of time
        interaction = inv_temp * log_time  # Interaction term
        
        X_kinetic = torch.tensor(np.hstack([X_vals, inv_temp, log_time, interaction]))
        
        # Solvent features
        if self.mixed:
            A = torch.tensor(self.featurizer.loc[X["SOLVENT A NAME"]].values)
            B = torch.tensor(self.featurizer.loc[X["SOLVENT B NAME"]].values)
            pct = torch.tensor(X["SolventB%"].values.reshape(-1, 1))
            
            if flip:
                # Symmetry flip: swap A and B
                X_chem = B * (1 - (1-pct)) + A * (1-pct)
            else:
                X_chem = A * (1 - pct) + B * pct
        else:
            X_chem = torch.tensor(self.featurizer.loc[X["SOLVENT NAME"]].values)
            
        return torch.cat([X_kinetic, X_chem], dim=1)

In [4]:
# MLP with BatchNorm and Dropout
class MLPInternal(nn.Module):
    def __init__(self, input_dim):
        super(MLPInternal, self).__init__()
        self.net = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 3),
            nn.Sigmoid()  # Yields are 0-1
        )

    def forward(self, x):
        return self.net(x)

In [5]:
# Symmetric Bagged Model
class SymmetricBaggedModel(nn.Module):
    def __init__(self, data='single', n_models=3, epochs=200):
        super().__init__()
        self.data_type = data
        self.featurizer = KineticMixingFeaturizer(mixed=(data=='full'))
        self.n_models = n_models
        self.epochs = epochs
        self.models = nn.ModuleList()

    def train_model(self, X_train, y_train):
        # Standard data
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            # Augmented data (train on both symmetries)
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            model = MLPInternal(input_dim).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)
            criterion = nn.HuberLoss()  # Robust to outliers
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=20
            )
            
            for epoch in range(self.epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        # Test Time Augmentation for mixed solvents
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    p1 = model(X_std)
                    p2 = model(X_flip)
                    pred_sum += (p1 + p2) * 0.5
            
            avg_pred = pred_sum / self.n_models
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models

        return avg_pred.cpu()

In [6]:
# Test with a quick run on single solvent
print("Testing single solvent task...")
X, Y = load_data("single_solvent")
print(f"Single solvent data: {X.shape}, {Y.shape}")
print(f"Solvents: {X['SOLVENT NAME'].nunique()}")

Testing single solvent task...
Single solvent data: (656, 3), (656, 3)
Solvents: 24


In [7]:
########### TASK 0: Single Solvent ###########
print("\n--- TASK 0: Single Solvent ---")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = SymmetricBaggedModel(data='single', n_models=3, epochs=200)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X).numpy()
    
    # Store for CV score calculation
    all_actuals.append(test_Y.values)
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })

submission_single_solvent = pd.DataFrame(all_predictions)

# Calculate CV score for single solvent
all_actuals_np = np.vstack(all_actuals)
all_preds_np = np.array([[p['target_1'], p['target_2'], p['target_3']] for p in all_predictions])
single_mse = np.mean((all_actuals_np - all_preds_np) ** 2)
print(f"\nSingle Solvent CV MSE: {single_mse:.6f}")


--- TASK 0: Single Solvent ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:25<09:53, 25.78s/it]

  8%|▊         | 2/24 [00:50<09:12, 25.11s/it]

 12%|█▎        | 3/24 [01:14<08:33, 24.45s/it]

...

 92%|█████████▏| 22/24 [09:09<00:50, 25.11s/it]

 96%|█████████▌| 23/24 [09:34<00:25, 25.09s/it]

100%|██████████| 24/24 [09:59<00:00, 25.09s/it]

100%|██████████| 24/24 [09:59<00:00, 24.96s/it]


Single Solvent CV MSE: 0.010916


In [8]:
########### TASK 1: Full Data (Mixtures) ###########
print("\n--- TASK 1: Full Data (Mixtures) ---")
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = SymmetricBaggedModel(data='full', n_models=3, epochs=200)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X).numpy()
    
    # Store for CV score calculation
    all_actuals_full.append(test_Y.values)
    
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({
            "task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })

submission_full_data = pd.DataFrame(all_predictions_full)

# Calculate CV score for full data
all_actuals_full_np = np.vstack(all_actuals_full)
all_preds_full_np = np.array([[p['target_1'], p['target_2'], p['target_3']] for p in all_predictions_full])
full_mse = np.mean((all_actuals_full_np - all_preds_full_np) ** 2)
print(f"\nFull Data CV MSE: {full_mse:.6f}")


--- TASK 1: Full Data (Mixtures) ---


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [01:27<17:31, 87.59s/it]

 15%|█▌        | 2/13 [02:53<15:50, 86.39s/it]

 23%|██▎       | 3/13 [04:21<14:30, 87.06s/it]

...

 85%|████████▍ | 11/13 [16:07<03:00, 90.44s/it]

 92%|█████████▏| 12/13 [17:41<01:31, 91.56s/it]

100%|██████████| 13/13 [19:15<00:00, 92.15s/it]

100%|██████████| 13/13 [19:15<00:00, 88.85s/it]


Full Data CV MSE: 0.011510


In [ ]:
# Combine and save submission
submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

# Calculate overall CV score (weighted average based on sample counts)
total_samples = len(all_actuals_np) + len(all_actuals_full_np)
overall_mse = (single_mse * len(all_actuals_np) + full_mse * len(all_actuals_full_np)) / total_samples

print(f"\n=== FINAL RESULTS ===")
print(f"Single Solvent MSE: {single_mse:.6f}")
print(f"Full Data MSE: {full_mse:.6f}")
print(f"Overall CV MSE: {overall_mse:.6f}")
print(f"\nSubmission saved to /home/submission/submission.csv")
print(f"Submission shape: {submission.shape}")